In [1]:
import pandas as pd
import re
import numpy as np
import json
import sqlite3
from ollama_tools import ask_llm
import fitz
import os
from ollama import chat
from ollama import Client
from ollama import ChatResponse
from ollama import generate

In [2]:
client = Client(host='http://localhost:11434')

In [3]:
file_path = "data-extration.xlsx"
xls = pd.ExcelFile(file_path)
rq_sheets = ["RQ2", "RQ3_Systems", "RQ4_Learning", "RQ5_Partition_Method", "RQ5_Metric"]

def extract_citation_key(cite):
    if isinstance(cite, str):
        match = re.search(r"\\cite{(.+?)}", cite)
        return match.group(1) if match else cite
    return cite

processed_sheets = {}
for sheet in rq_sheets:
    df = xls.parse(sheet)
    df.columns = [str(col).strip().lower() for col in df.columns]
    
    df = df.loc[:, ~df.columns.str.contains('^unnamed')]
    df = df.loc[:, ~df.columns.str.contains('^comentário')]
    df = df.loc[:, ~df.columns.str.contains('^obs:')]
    
    df = df.dropna(how='all')
    
    if "reference" in df.columns:
        df["reference"] = df["reference"].apply(extract_citation_key)
        
        agg_dict = {}
        for col in df.columns:
            if col != "reference":
                agg_dict[col] = lambda x: list(x.dropna().unique()) if len(x.dropna()) > 0 else np.nan
        
        df = df.groupby("reference", as_index=False).agg(agg_dict)
        
        for col in df.columns:
            if col != "reference":
                df[col] = df[col].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x)

    processed_sheets[sheet] = df

merged_df = processed_sheets[rq_sheets[0]]
for sheet in rq_sheets[1:]:
    merged_df = merged_df.merge(processed_sheets[sheet], on="reference", how="outer")

merged_df.columns = [col.split('.')[-1] if '.' in col else col for col in merged_df.columns]

merged_df.drop(columns=["related", "ref", "cite"], inplace=True)
merged_df



,reference,sampling method,systemname,domain,nfp,strategy,dataset,technique,partition method,evaluation metric
0,Alshehri2023,reamostragem SMOTE,Eclipse,system files,NaN,NaN,NaN,"[AdaBoost com J48, J48]",Validação cruzada,"[Recall , Precision, Medida F ]"
1,Alves2020,"[Coverage-based , Solver-based, Randomized ...",x264,NaN,"Tempo de codificação e codificação, tamanho",NaN,https://github.com/jualvespereira/ICPE2020,regressão linear múltipla,NI,MRE
2,Arcaini2020,Não identifiquei,No identificado,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ballesteros2021,NI,"[x264, Wget, Berkeley DB Memory, Sensor Networ...",NaN,Population Size / Archive Size: 400; Number of...,SI,NaN,regressão linear,NI,Coverage Metric (CM)
4,Chen2022,amostragem adaptativa com d-Simplexed,Spark,Database System,"Count,Executor Memory,Executor Threads, Memory...",NaN,NaN,Rede Neural Multicamadas (NN),Bootstrap,MAPE
...,...,...,...,...,...,...,...,...,...,...
59,tipu2022:cc,Random Sampling,"[MPI-I/O, SEG-Y I/O]",NaN,"[Number of MPI node, MPI processes per node, S...",NaN,NaN,Artificial Neural Networks,NaN,"[accuracy, MSE, MAE, MAPE]"
60,valov2020:icpe,amostragem pseudoaleatória,"[BZIP2, GZIP, XZ, FLAC, x264]",NaN,NaN,NaN,NaN,"[árvores de regressão, Regressão linear simple]",NaN,"[MAPE, LOOCV]"
61,vitui2021:ese,Amostragem aleatória,"[Open-Src, Entprz. 1, Entprz. 2]",NaN,NaN,NaN,NaN,"[Random Forest, XGBoost trees, Multi-Layer Per...","[cross-validation, validação cruzada leave-one...","[Median Percentage Deviation, MAPE, MAE, (MSE,..."
62,yufei2024:jss,"[Random Sampling, Neighborhood Sampling, Input...","[SQLite, BDB-C, BDB-J, LLVM, Sac, Apache, x264...","[Database, Compiler, Web Server, Video Encoder...","[Execution Time, Response Time, Video Encoding...",Execution,https://github.com/RSFIN/RSFIN/tree/master/data,"[Artificial Neural Networks (ANN), Deep Learni...",NaN,NaN


In [4]:
merged_df[merged_df["reference"] == "lesoil2024"]

,reference,sampling method,systemname,domain,nfp,strategy,dataset,technique,partition method,evaluation metric
46,lesoil2024,"[K-means, HDBScan, Amostragem aleatória, Submo...","[gcc, ImageMagick, lingeling, nodeJS, poppler,...","[.c programs, images, SAT formulae, .js script...","[size, ctime, exec, size, time, #confl.,#reduc...",EX,NaN,"[OLS Regression, Desicion Tree, Random forest,...",NI,Mean Absolute Percentage Error (MAPE)


In [5]:
merged_df["reference"] = merged_df["reference"].str.replace(":", "_", regex=False)

In [6]:
def build_message(topic: str, context: str, question: str, some_answer_examples: str,answer_prefix: str, base_text: str, text_example:str, answer_text_example:str, model: str) -> str:
    message = [{
        'role': "system",
        'content': f"""You are an expert scientific article analyzer. Your task is to extract specific information 
        from scientific texts based on provided questions and context. When asked about {topic}, understand that {context}
        Examples include: {some_answer_examples}
        Your answer should be concise and directly address the question based on the provided text, starting with the phrase:{answer_prefix}"""
    },
    {
        'role': "user",
        'content': f"""Please answer the question: {question}, based on the following text: {base_text}"""
    },{
        'role': "assistant",
        'content': f"""Question: {question}\n base_text: {text_example}\nAnswer: {answer_text_example}
        """
    }]

    return message
    
    

In [7]:
references_list = merged_df["reference"].tolist()

In [ ]:
evaluated_metrics = ["sampling method", "dataset", "technique", "partition method", "evaluation metric"]

In [ ]:
# def extract_citation_key(cite):
#     match = re.search(r"\\cite{(.+?)}", cite)
#     return match.group(1) if match else cite

# df["reference"] = df["reference"].apply(extract_citation_key)

# result_dict = {}

# for _, row in df.iterrows():
#     ref = row["reference"]
#     result_dict[ref] = {
#         "sampling method": row["sampling method"],
#         "dataset": row["dataset"],
#         "technique": row["technique"],
#         "partition method": row["partition method"],
#         "evaluation metric": row["evaluation metric"]
#     }

# result_dict

KeyError: 'sampling method'

In [11]:
local_llm_models = ["deepseek-r1:32b", "magistral:24b", "llama4:latest"] #testar o llama 4

In [12]:
def create_llm_tables(db_path: str, llm_models: list):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS llm_models (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            model_name TEXT NOT NULL
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS llm_responses (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            model_id INTEGER,
            pdf_name varchar(255),
            prompt_technique varchar(255),
            text_segmentation_stategy varchar(255),
            metric varchar(255),
            response TEXT,
            FOREIGN KEY (model_id) REFERENCES llm_models(id)
        )
    ''')
    
    for model in llm_models:
        cursor.execute('INSERT INTO llm_models (model_name) VALUES (?)', (model,))
    
    conn.commit()
    conn.close()

def insert_llm_response(db_path: str, model_name: str, pdf_name: str, prompt_technique: str, metric: str, response: str, text_segmentation_stategy: str):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute('SELECT id FROM llm_models WHERE model_name = ?', (model_name,))
    model_id = cursor.fetchone()
    if not model_id:
        cursor.execute('INSERT INTO llm_models (model_name) VALUES (?)', (model_name,))
        conn.commit()
        cursor.execute('SELECT id FROM llm_models WHERE model_name = ?', (model_name,))
        model_id = cursor.fetchone()
    
    if model_id:
        cursor.execute('''
            INSERT INTO llm_responses (model_id, pdf_name, prompt_technique, metric, response, text_segmentation_stategy)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (model_id, pdf_name, prompt_technique, metric, response, text_segmentation_stategy))
        
        conn.commit()
    else:
        print(f"Model {model_name} not found in the database.")
    
    conn.close()

In [13]:
def find_all_paths_and_names(folder_path):
    all_paths = []
    all_names = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            all_paths.append(os.path.join(root, file))
            all_names.append(file)
    return all_paths, all_names

In [ ]:
def simple_extraction(path: str) -> dict[int, str]:
        """This function extracts text from a PDF file and returns the text along with the number of pages.

        Args:
            doc (fitz.Document): The PDF document object

        Returns:
            tuple[str, int]: The text inside the pdf(without any cleaning) and page count
        """
        try:
            text = ""
            total_pages = 0
            doc = fitz.open(path)
            for idx, page in enumerate(doc):
                text += f"<--page_start:{idx+1}-->"
                text +=  page.get_text()
                text += f"<--page_end:{idx+1}-->"
                total_pages += 1
            return text, total_pages

        except Exception as e:
            print(f"An error occurred: {e} in {__file__} ")
            return "", 0

In [18]:
def update_database_with_text_segments(db_path: str):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute("SELECT id, pdf_name FROM pdfs")
    pdfs = cursor.fetchall()
    
    all_paths, all_names = find_all_paths_and_names("/home/pramos/Documents/AutoSLR/papers_pdf")
    
    for pdf_id, pdf_name in pdfs:
        print(f"Processing PDF: {pdf_name} (ID: {pdf_id})")
        
        pdf_path = None
        for path, name in zip(all_paths, all_names):
            if name == pdf_name:
                pdf_path = path
                break
        
        if not pdf_path:
            print(f"PDF file not found for: {pdf_name}")
            continue
        
        try:
            full_text, count = simple_extraction(pdf_path)
        except Exception as e:
            print(f"Error extracting text from {pdf_name}: {e}")
            continue
        
        cursor.execute("""
            SELECT id, position, section_number, section_title 
            FROM extracted_text 
            WHERE pdf_id = ? 
            ORDER BY position ASC
        """, (pdf_id,))
        
        sections = cursor.fetchall()
        
        if not sections:
            print(f"No sections found for PDF: {pdf_name}")
            continue
        
        # Atualizar o content de cada seção
        for i, (section_id, start_pos, section_number, section_title) in enumerate(sections):
            try:
                if i + 1 < len(sections):
                    end_pos = sections[i + 1][1]
                else:
                    end_pos = len(full_text)
                
                section_content = full_text[start_pos:end_pos].strip()
                
                cursor.execute("""
                    UPDATE extracted_text 
                    SET content = ? 
                    WHERE id = ?
                """, (section_content, section_id))
                
                print(f"  Updated section {section_number}: {section_title[:50]}... "
                      f"(chars: {start_pos}-{end_pos}, length: {len(section_content)})")
                
            except Exception as e:
                print(f"Error processing section {section_number} of {pdf_name}: {e}")
                continue
        
        conn.commit()
        print(f"Completed processing {pdf_name}")
    
    conn.close()
    print("Database update completed!")




    
update_database_with_text_segments("/home/pramos/Documents/AutoSLR/validations/regex_validation/results/extern_llm-gemini.db", )

Processing PDF: Alshehri2020.pdf (ID: 21)
Error extracting text from Alshehri2020.pdf: name '__file__' is not defined
Processing PDF: Arcaini2020.pdf (ID: 16)
Error extracting text from Arcaini2020.pdf: name '__file__' is not defined
Processing PDF: Chen2022.pdf (ID: 22)
Error extracting text from Chen2022.pdf: name '__file__' is not defined
Processing PDF: Damasceno2021.pdf (ID: 23)
Error extracting text from Damasceno2021.pdf: name '__file__' is not defined
Processing PDF: Dorn2023ese.pdf (ID: 77)
Error extracting text from Dorn2023ese.pdf: name '__file__' is not defined
Processing PDF: Gao2021-ICSE.pdf (ID: 24)
Error extracting text from Gao2021-ICSE.pdf: name '__file__' is not defined
Processing PDF: García2021.pdf (ID: 25)
Error extracting text from García2021.pdf: name '__file__' is not defined
Processing PDF: H. Martin2022.pdf (ID: 17)
Error extracting text from H. Martin2022.pdf: name '__file__' is not defined
Processing PDF: Ha-2019ICSME.pdf (ID: 26)
Error extracting text from

In [ ]:
def find_segment_text(text: str, segment_names: list) -> str:


In [ ]:
#1) local llm tests
db_path = "llm_evaluation.db"
create_llm_tables(db_path, local_llm_models)

for model in local_llm_models:
    print(f"Evaluating with model: {model}")
    for path, name in zip(all_file_paths, all_file_names):
        
        text = extract_text_from_pdf(path)
        segmented_text = adfas()

        for metric in evaluated_metrics:
            prompt_zero_shot = get_prompt_by_metric_zero_shot(metric)
            prompt_few_shot = get_prompt_by_metric_few_shot(metric)
            
            answer_zero_shot = ask_llm(prompt_zero_shot, [segmented_text], model=model)
            answer_few_shot = ask_llm(prompt_few_shot, [segmented_text], model=model)
            
            insert_llm_response(db_path, model, name, "zero_shot", metric, answer_zero_shot)
            insert_llm_response(db_path, model, name, "few_shot", metric, answer_few_shot)

        


In [ ]:
#2) remote llm tests
db_path = "llm_evaluation.db"
create_llm_tables(db_path, ["gemini-2.5-flash-preview-05-20", ]) #make sense use "gemini-2.0-flash", "gemini-2.0-flash-lite"?